In [56]:
import json_lines
import gzip
import json
import traceback
import sys

from soft404.utils import cleaned_selector, selector_to_text, get_text_blocks


with gzip.open('../text_items.jl.gz', 'wt') as outf:
    with json_lines.open('../crawler/items.jl.gz', broken=True) as f:
        max_errors = 10
        n_errors = 0
        for item in f:
            try:
                sel = cleaned_selector(item.pop('html'))
                text_item = {
                    'url': item['url'],
                    'text': selector_to_text(sel),
                    'title': ' '.join(sel.xpath('/html/head/title//text()').extract()),
                    'status': item['status'],
                }
                body = sel.xpath('/html/body')
                if body:
                    text_item['blocks'] = get_text_blocks(body[0].root)
            except Exception:
                n_errors += 1
                if n_errors <= max_errors:
                    print('Error processing item {}'.format(item['url'], file=sys.stderr))
                    traceback.print_exc()
            else:
                outf.write(json.dumps(text_item))
                outf.write('\n')
print('Number of erorrs: {}'.format(n_errors))

Error processing item http://www.ecns.cn/app/ecns/index.shtml


Traceback (most recent call last):
  File "/Users/kostia/shub/memex/soft404/soft404/utils.py", line 37, in cleaned_selector
    tree = _cleaned_html_tree(html)
  File "/Users/kostia/shub/memex/soft404/soft404/utils.py", line 27, in _cleaned_html_tree
    tree = lxml.html.fromstring(html.encode('utf8'), parser=parser)
UnicodeEncodeError: 'utf-8' codec can't encode character '\udd7d' in position 3255: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-56-96997bfac119>", line 16, in <module>
    sel = cleaned_selector(item.pop('html'))
  File "/Users/kostia/shub/memex/soft404/soft404/utils.py", line 42, in cleaned_selector
    sel = parsel.Selector(html)
  File "/Users/kostia/shub/memex/soft404/venv/lib/python3.5/site-packages/parsel/selector.py", line 151, in __init__
    root = self._get_root(text, base_url)
  File "/Users/kostia/shub/memex/soft404/venv/lib/python3.5/site-packages/parsel/

Number of erorrs: 1


In [20]:
import json_lines

with json_lines.open('../crawler/items.jl.gz', broken=True) as f:
    for i, item in enumerate(f):
        if i > 100:
            break

In [21]:
from soft404.utils import _cleaned_html_tree
tree = _cleaned_html_tree(item['html'])
sel = parsel.Selector(root=tree, type='html')

In [31]:
# this is not quite what I want - I want to avoid breaking text on inline nodes
len(list(x for x in (x.strip() for x in tree.itertext()) if x))

212

In [37]:
from soft404.utils import get_text_blocks

text_blocks = get_text_blocks(tree)
len(text_blocks), text_blocks

(154,
 [('', 'VK on the App Store'),
  ('li', 'Open Menu Close Menu'),
  ('li', 'Apple'),
  ('li', 'Shopping Bag'),
  ('li', 'Apple'),
  ('li', 'Mac'),
  ('li', 'iPad'),
  ('li', 'iPhone'),
  ('li', 'Watch'),
  ('li', 'TV'),
  ('li', 'Music'),
  ('li', 'Support'),
  ('li', 'Search apple.com'),
  ('li', 'Shopping Bag'),
  ('li', 'Overview'),
  ('li', 'Music'),
  ('li', 'Video'),
  ('li', 'Charts'),
  ('div',
   "Opening the iTunes Store. If iTunes doesn't open, click the iTunes application icon in your Dock or on your Windows desktop. Progress Indicator"),
  ('div',
   "Opening the iBooks Store. If iBooks doesn't open, click the iBooks app in your Dock. Progress Indicator"),
  ('div', 'iTunes'),
  ('h2',
   "iTunes is the world's easiest way to organize and add to your digital media collection."),
  ('p',
   'We are unable to find iTunes on your computer. To download the free app VK by VKontakte, get iTunes now.'),
  ('p',
   'Already have iTunes? Click I Have iTunes to open it now. iTu